In [60]:
import pandas as pd
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools
import pygsheets as pygs
from datetime import timedelta, date
from pandas.tseries.offsets import MonthEnd

In [61]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [73]:
g_drive_cred = r'C:\Users\darius kay\Dev\gitlab\test\python\credentials.json'
# Main dashboard
workbook1 = "https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414"
# Reason dashboard
workbook2 = "https://docs.google.com/spreadsheets/d/1mZUH-iNi3N2uAYlCUP-sH47hlGZxSy-yIJ-NHz5oaVk/edit#gid=408816025"
# Style dashboard.
workbook3 = 'https://docs.google.com/spreadsheets/d/1oV3y4aTKGDEmllsvn4Aq3mMPqJeA3F2yko8JX8tVIIg/edit#gid=555002914'

# Function to clear data range.
def clear_data_range(workbook_name, tab_name, data_range):
    print(g_drive_cred)
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)
    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)
    # select the first sheet
    wks = ws.worksheet_by_title(tab_name)
    print(ws.id)
    wks.clear()
    print("Cleared")
    return

def pd_to_gsheets(workbook_name, sheet_name, df, row_num, col_num):
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)

    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)

    # select the first sheet
    wks = ws.worksheet_by_title(sheet_name)

    # update the designated sheet with df, starting at cell B2.
    wks.set_dataframe(df, (row_num, col_num))

    return

In [74]:
def date_checker(view, date, schedule):
    if view == 'forecast':
        if schedule == 'week':
            try:
                return date + timedelta(days=21)
            except TypeError:
                return
    else:
        return date

In [51]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [81]:
current_eow = datetime.date.today() - timedelta(days=datetime.date.today().weekday()) + timedelta(days=6)

starting_55_eow = current_eow - timedelta(weeks=58)
starting_55_eow.strftime("%Y-%m-%d")

'2019-12-01'

In [52]:
# Primary weekly forecast query.
weekly_forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 AND fulfillment_week_ended <= CAST(DATE_TRUNC('week',GETDATE())+INTERVAL '1 weeks' -1 AS DATE)"

weekly_forecast_category_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, master_category, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 AND a.week_ended <= CAST(DATE_TRUNC('week',GETDATE())+INTERVAL '1 weeks' -1 AS DATE)"

weekly_forecast_reason_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 AND a.week_ended <= CAST(DATE_TRUNC('week',GETDATE())+INTERVAL '1 weeks' -1 AS DATE)"

# Actual weekly primary query.
weekly_actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

weekly_actual_category_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, master_category, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

weekly_actual_reason_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

In [53]:
# Monthly actual primary query.
monthly_actual_query = "SELECT fulfillment_month_ended month_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

monthly_actual_category_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_month_ended month_ended, %s, master_category, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

monthly_actual_reason_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_month_ended month_ended, %s, master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

# Primary monthly forecast query.
monthly_forecast_query = "SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', LAST_DAY(fulfillment_week_ended + 21)) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

monthly_forecast_category_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, master_category, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

monthly_forecast_reason_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1"

In [54]:
# Weekly Forecasted qty returned/exchanged query.
weekly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND plant IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

weekly_forecast_qty_style = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Monthly Forecasted qty returned/exchanged query.
monthly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND plant IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

monthly_forecast_qty_style = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN send_back__type in %s AND plant IN %s THEN 1 ELSE 0 END)"

actual_qty_style = "SUM(CASE WHEN send_back__type in %s THEN 1 ELSE 0 END)"

In [55]:
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product"

# Edit for cushion only non-cover.
product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product"

In [56]:
sizes = "CASE WHEN size ILIKE '3%x%5%' THEN '3x5' WHEN size ILIKE '8%r%' THEN '8-Round' WHEN size ILIKE '6%r%' THEN '6-Round' WHEN size ILIKE '2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '9%x%12%' THEN '9x12' WHEN size ILIKE '8%x%10%' THEN '8x10' WHEN size ILIKE '5%x%7%' THEN '5x7' WHEN size ILIKE '2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '6%x%9%' THEN '6x9' END size"

In [57]:
query_params = {
    'send_back_product_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'send_back_product_monthly': {'view': {'actual': [monthly_actual_query, actual_qty], 'forecast': [monthly_forecast_query, monthly_forecast_qty]},
                          'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                          'send_back__type': {'send_back_both': "('return', 'exchange')",
                                              'return': "('return')", 'exchange': "('exchange')"},
                            'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'send_back_product_size_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'send_back_product_size_monthly': {'view': {'actual': [monthly_actual_query, actual_qty], 'forecast': [monthly_forecast_query, monthly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'send_back_product_category_weekly': {'view': {'actual': weekly_actual_category_query, 'forecast': weekly_forecast_category_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
#                                 'size': {'all': "'all' size", 'size_categories': sizes},
#                                 'master_category': {'all': "'all' master_category", 'category': "master_category"}
                                         },
    'send_back_product_category_monthly': {'view': {'actual': monthly_actual_category_query, 'forecast': monthly_forecast_category_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
#                                 'size': {'all': "'all' size", 'size_categories': sizes},
#                                 'master_category': {'all': "'all' master_category", 'category': "master_category"}
                                          },
    'send_back_product_size_reason_weekly': {'view': {'actual': weekly_actual_reason_query, 'forecast': weekly_forecast_reason_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}
#                                 'master_reason': {'all': "'all' master_reason", 'category': "master_reason"}
                                            },
    'send_back_product_size_reason_monthly': {'view': {'actual': monthly_actual_reason_query, 'forecast': monthly_forecast_reason_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}
#                                 'master_reason': {'all': "'all' master_reason", 'category': "master_reason"}
                                             },
    'send_back_style_weekly': {'view': {'actual': [weekly_actual_query, actual_qty_style], 'forecast': [weekly_forecast_query, weekly_forecast_qty_style]},
                              'rug_name': {'rug_name': " design_name rug_name"},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}},
    'send_back_style_monthly': {'view': {'actual': [monthly_actual_query, actual_qty_style], 'forecast': [monthly_forecast_query, monthly_forecast_qty_style]},
                              'rug_name': {'rug_name': " design_name rug_name"},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}}
}

In [59]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
    print(key)
    tab_df = pd.DataFrame()
    master_query = ""
    combinations = []
    check = []
    keys = value.keys()
    values = (value[key] for key in keys)
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    values2 = (value[key] for key in keys)
    check.append(dict(zip(keys, combination)) for combination in itertools.product(*values2))
    for c in check:
        total = len(list(c))
    for combination in combinations:
        for item, combo in enumerate(combination):
            keys = list(keys)
#             print(keys)
#             print(item+1)
            grouping = ""
            for i in range(len(keys)):
#                 print(query_params[key][keys[i]][combo[keys[i]]])
                if 'category' in key or 'reason' in key:
                    if i == 0:
                        query = query_params[key][keys[i]][combo[keys[i]]]
                        params = f"'{combo[keys[i]]}' AS {keys[i]}"
#                         print(params)
                    elif keys[i] == 'send_back__type' or keys[i] == 'plant':
                        if keys[i] == 'send_back__type':
                            send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
#                             print(send_back_type)
                        else:
                            plant = str(query_params[key][keys[i]][combo[keys[i]]])
#                             print(plant)
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
#                         print(params)

                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"

                else:
                    grouping = f"{grouping}, {i+2}"
    #                 print(keys[i], grouping)
                    if i == 0:
                        query = query_params[key][keys[i]][combo[keys[i]]][0]
    #                     print(query)
                        calculation = query_params[key][keys[i]][combo[keys[i]]][1]
    #                     print(calculation)
                        params = f"'{combo[keys[i]]}' AS {keys[i]}"
    #                     print(params)
                    else:
                        if keys[i] == 'send_back__type':
                            if 'style' in key:
                                send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                                calculation = calculation %(send_back_type)
                                params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                            else:
                                send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                                params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                        elif keys[i] == 'plant':
                            calculation = calculation %(send_back_type, str(query_params[key][keys[i]][combo[keys[i]]]))
                            params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                        else:
#                             if 'style' in key and keys[i] == 'send_back__type':
#                                 send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
#                                 calculation = calculation %(send_back_type)
                            params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
# #             print(keys)
            if 'reason' in key:
                query = query %(params, send_back_type, params, send_back_type)
            elif 'category' in key:
                query = query %(params, send_back_type, plant, params, send_back_type, plant)
            else:
                query = query %(params, calculation, calculation, grouping)
#             print(query)
#             print(calculation)
#             print(params)
#             print(grouping)
#             df = pd.read_sql(query, conn)
#             tab_df = tab_df.append(df)
    
            if item+1 == total:
                master_query = f"{master_query} UNION {query} ORDER BY 1 DESC;"
            else:
                if item == 0:
                    master_query = query
                else:
                    master_query = f"{master_query} UNION {query}"
    
    print(master_query)
    
    tab_df = pd.read_sql(master_query, conn)
    
    if 'reason' in key:
        workbook = workbook2
    elif 'style' in key:
        workbook = workbook3
    else:
        workbook = workbook1

    if 'weekly' in key:
        # Add 21 days to week_ended for forecast only.
        tab_df['week_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['week_ended'], 'week'), axis=1)
        
        ### Add category to uid, just one
        if 'category' in key:
            keys.insert(0, 'week_ended')
            keys.append('master_category')
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
        else:    
            # Concat first four columns.
            keys.insert(0, 'week_ended')
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
            ### Add in second concat for master reason.
            if 'reason' in key:
                keys.append('master_reason')
                uid2 = tab_df[keys].astype(str).agg('_'.join, axis=1)
                tab_df.insert(loc=0, column='uid2', value=uid2)
        tab_df = tab_df[tab_df['week_ended'] <= current_eow]
        
    elif 'monthly' in key:
        # Add 21 days to month_ended for forecast only.
    #         tab_df['month_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['month_ended'], 'month'), axis=1)
        if 'week_ended' in tab_df.columns:
            tab_df = tab_df.rename(columns={'week_ended': 'month_ended'})
            
        ### Add category to uid, just one
        if 'category' in key:
            keys.insert(0, 'month_ended')
            keys.append('master_category')
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
        else:    
            # Concat first four columns.
            keys.insert(0, 'month_ended')
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
            ### Add in second concat for master reason.
            if 'reason' in key:
                keys.append('master_reason')
                uid2 = tab_df[keys].astype(str).agg('_'.join, axis=1)
                tab_df.insert(loc=0, column='uid2', value=uid2)

#     clear_data_range(workbook, key, "A1:J100000")
#     pd_to_gsheets(workbook, key, tab_df, 1, 1)

send_back_product_weekly
SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5 HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 UNION SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'LA' plant, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 

send_back_product_monthly
SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5 HAVING DATE_PART('year', fulfillment_month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 UNION SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'LA' plant, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP 

send_back_product_size_weekly
SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, 'all' size, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 UNION SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, CASE WHEN size ILIKE '3%x%5%' THEN '3x5' WHEN size ILIKE '8%r%' THEN '8-Round' WHEN size ILIKE '6%r%' THEN '6-Round' WHEN size ILIKE '2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '9%x%12%' THEN '9x12' WHEN size ILIKE '8%x%10%' THEN '8x10' WHEN size ILIKE '5%x%7%' THEN '5x7' WHEN si

send_back_product_size_monthly
SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, 'all' size, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4, 5, 6 HAVING DATE_PART('year', fulfillment_month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 UNION SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, CASE WHEN size ILIKE '3%x%5%' THEN '3x5' WHEN size ILIKE '8%r%' THEN '8-Round' WHEN size ILIKE '6%r%' THEN '6-Round' WHEN size ILIKE '2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '9%x%12%' THEN '9x12' WHEN size ILIKE '8%x%10%' THEN '8x10' WHEN size ILIKE '5%x%7%' THEN '5x7' W

send_back_product_category_weekly
SELECT a.week_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, master_category, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') GROUP

send_back_product_category_monthly
SELECT a.month_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A') GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' plant, master_category, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_category IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN ('return', 'exchange') AND plant IN ('LA', 'CHI', 'N/A')

send_back_product_size_reason_weekly
SELECT a.week_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' size, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN ('return', 'exchange') GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' size, master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN ('return', 'exchange') GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.vie

send_back_product_size_reason_monthly
SELECT a.month_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' size, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN ('return', 'exchange') GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view, 'all' product, 'send_back_both' send_back__type, 'all' size, master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN ('return', 'exchange') GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view

send_back_style_weekly
SELECT fulfillment_week_ended week_ended, 'actual' AS view,  design_name rug_name, 'send_back_both' send_back__type, SUM(CASE WHEN send_back__type in ('return', 'exchange') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4 HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 UNION SELECT fulfillment_week_ended week_ended, 'actual' AS view,  design_name rug_name, 'return' send_back__type, SUM(CASE WHEN send_back__type in ('return') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4 HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 1 UNION SELECT fulfillment_week_ended week_ended, 'actual' 

In [185]:
clear_data_range(workbook, key, "A1:J200000")
pd_to_gsheets(workbook, key, tab_df, 1, 1)

A1
J200000
C:\Users\darius kay\Dev\gitlab\test\python\credentials.json
1oV3y4aTKGDEmllsvn4Aq3mMPqJeA3F2yko8JX8tVIIg


In [13]:
print(key)

send_back_style_monthly


In [184]:
tab_df

,uid,week_ended,view,rug_name,send_back__type,qty_returned,qty_sold,rate
0,2021-01-03_actual_Sailor's Map_send_back_both,2021-01-03,actual,Sailor's Map,send_back_both,0,28,0.000000
1,2021-01-03_actual_Mariposa Polychrome_send_bac...,2021-01-03,actual,Mariposa Polychrome,send_back_both,0,42,0.000000
2,2021-01-03_actual_Malileh Heriz Opal_send_back...,2021-01-03,actual,Malileh Heriz Opal,send_back_both,1,46,0.021739
3,2021-01-03_actual_Darkside Damask Delft Blue_s...,2021-01-03,actual,Darkside Damask Delft Blue,send_back_both,0,96,0.000000
4,2021-01-03_actual_Rogue Squadron Dark Teal_sen...,2021-01-03,actual,Rogue Squadron Dark Teal,send_back_both,0,190,0.000000
...,...,...,...,...,...,...,...,...
54897,2019-02-03_forecast_Patchwork Boho_exchange,2019-02-03,forecast,Patchwork Boho,exchange,0,1,0.000000
54898,2019-02-03_forecast_Other_exchange,2019-02-03,forecast,Other,exchange,0,456,0.000000
54899,2019-02-03_forecast_nan_exchange,2019-02-03,forecast,None,exchange,0,887,0.000000
54900,2019-01-27_forecast_Other_exchange,2019-01-27,forecast,Other,exchange,0,5,0.000000


## Weekly Category
* From Oct 1 - Now: 158,000+ rows

In [ ]:
# category_weekly = tab_df
# category_weekly
# category_weekly[(category_weekly['week_ended'] > datetime.date(year=2020,month=10,day=1)) & (category_weekly['week_ended'] < datetime.date(year=2021,month=1,day=4))]
clear_data_range(workbook1, key, "A1:K500000")
pd_to_gsheets(workbook1, key, category_weekly, 1, 1)

## Category Monthly
*

In [ ]:
# category_monthly = tab_df
# category_monthly
# category_monthly[(category_monthly['month_ended'] > datetime.date(year=2020,month=10,day=1)) & (category_monthly['month_ended'] < datetime.date(year=2021,month=1,day=4))]
clear_data_range(workbook1, key, "A1:K500000")
pd_to_gsheets(workbook1, key, category_monthly, 1, 1)